In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.svm import LinearSVC

plt.ion()   # interactive mode

data_dir = 'drive/My Drive/dataset/'
data_transforms = {
    
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    
    "test": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in ["train",  "test"]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=4
    )
    for x in ["train",  "test"]
}

dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "test"]}
#for x in ["train",  "test"]:
 #   print("Loaded {} images under {}".format(dataset_sizes[x], x))

#print("Classes: ")
class_names = image_datasets["train"].classes
#print(image_datasets["train"].classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
def calculate_svm(model):
  
    model.eval()
    model.to(device)
    
    feat_train = []
    feat_classes_train = []
    feat_test = []
    feat_classes_test = []
    
    with torch.no_grad():
        for i, (inputs,classes) in enumerate(dataloaders['train']):
            inputs = inputs.to(device)
            outputs = model(inputs)
            feat_train.extend(outputs.cpu().numpy())
            feat_classes_train.extend(classes.cpu().numpy())
            
    with torch.no_grad():
        for i, (inputs,classes) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            outputs = model(inputs)
            feat_test.extend(outputs.cpu().numpy())
            feat_classes_test.extend(classes.cpu().numpy())
            
    clf_linear = LinearSVC(random_state=0, max_iter=1000)
    classifier = clf_linear.fit(feat_train, feat_classes_train)

    
    print('Accuracy : {:.2f}'.format(100*clf_linear.score(feat_test,feat_classes_test)))
    return (feat_train, feat_classes_train, feat_test, feat_classes_test, clf_linear)
    
    
        
vgg16 = models.vgg16(pretrained=True)   
for param in vgg16.features.parameters():
    param.requires_grad = False
vgg16.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-2])


result_arrays = calculate_svm(vgg16)



In [0]:

def calculate_classbased_accuracies(feat_train, feat_classes_train, feat_test, feat_classes_test, clf):
    predictions = clf.predict(feat_test)
    test_number = dataset_sizes['test']/len(image_datasets["test"].classes)
    
    
    true_pred = np.zeros(len(image_datasets["test"].classes))
    for i in range(len(predictions)):
      if(predictions[i] == feat_classes_test[i]):
        true_pred[predictions[i]] = true_pred[predictions[i]] + 1
    for i in range(len(true_pred)):
      true_pred[i] = 100* true_pred[i]/test_number
      print("Class based accuracy for {} = {:.1f}%".format(class_names[i], true_pred[i]))
    
    

calculate_classbased_accuracies(result_arrays[0],result_arrays[1], result_arrays[2], result_arrays[3],result_arrays[4])

Class based accuracy for airport_inside = 60.0%
Class based accuracy for bar = 76.0%
Class based accuracy for bedroom = 72.0%
Class based accuracy for casino = 88.0%
Class based accuracy for inside_subway = 88.0%
Class based accuracy for kitchen = 84.0%
Class based accuracy for livingroom = 80.0%
Class based accuracy for restaurant = 60.0%
Class based accuracy for subway = 84.0%
Class based accuracy for warehouse = 96.0%
